In [10]:
import os
import cv2
import face_recognition
import pickle
import string

LABELS_TO_FACES_DICT_PICKLED_FILENAME = 'labels_to_face_encodings_dict.pkl'
os.chdir(r'C:\Users\yaz_s\OneDrive\Education_Stuff\Graduation_Project\Project_Files\Environment_Server')
print(os.getcwd())

C:\Users\yaz_s\OneDrive\Education_Stuff\Graduation_Project\Project_Files\Environment_Server


In [11]:
# Load face encodings and labels from photos

def load_mappings():
    if os.path.isfile(LABELS_TO_FACES_DICT_PICKLED_FILENAME):
        labels_to_encodings = pickle.load(open(LABELS_TO_FACES_DICT_PICKLED_FILENAME,'rb'))
    else:
        labels_to_encodings = {}
    
    return labels_to_encodings

def dump_mappings(labels_to_encodings):
    pickle.dump(labels_to_encodings,open(LABELS_TO_FACES_DICT_PICKLED_FILENAME,'wb'))
    return

def load_face_encodings_from_photos(directory):
    label_to_encodings = load_mappings()

    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            label = str(os.path.splitext(filename)[0]).lower()
            label = ''.join([letter for letter in label if letter in string.ascii_letters or letter in '_'])
            image_path = os.path.join(directory, filename)

            # Load image and generate face encoding 
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)
            print(bool(encoding),end=' ')
            if len(encoding) > 0:
                if label not in label_to_encodings.keys(): #Do not overwrite existing labels
                    label_to_encodings[label] = encoding[0]

    dump_mappings(label_to_encodings)

    return label_to_encodings



In [12]:
import time


# Directory containing face photos
photo_directory = 'Known_Faces'

# Load face encodings and labels from photos
labels_to_encodings = load_face_encodings_from_photos(photo_directory)

WIDTH = HEIGHT = 700

# Initialize webcam
video_capture = cv2.VideoCapture(0)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT,WIDTH)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH,HEIGHT)



known_face_labels, known_face_encodings = list(labels_to_encodings.keys()) , list(labels_to_encodings.values())


try:
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        
        start_time = time.time()
        # Find all face locations and encodings in the current frame
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)
        
        i = 0

        for encoding in face_encodings:
            # Compare the face encoding with known faces
            matches = face_recognition.compare_faces(known_face_encodings, encoding)
            name = "Unknown"
            

            if True in matches:
                
                first_match_index = matches.index(True)
                name = known_face_labels[first_match_index]
                    
            # Display the name near the face
            if face_locations:
                cv2.putText(frame, name, (face_locations[i][3] + 10 , face_locations[i][0] + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
                i = i + 1

        end_time = time.time()
        elapsed_time = end_time - start_time
        cv2.putText(frame, str(elapsed_time), (WIDTH//2, HEIGHT//4),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        
        # Display the resulting image
        cv2.imshow('Video', frame)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close OpenCV windows
    video_capture.release()
    cv2.destroyAllWindows()

finally:
    pass



True True 